### SPI 

#### data split 

In [1]:
import pandas as pd
path = '../data/spi10_2nrs.csv'
df = pd.read_csv(path)

In [2]:
from dataset import nrs2class 

df.pacu_nrs = df.pacu_nrs.map(lambda x: nrs2class(x,pain_type=2))

from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.4, shuffle=True, stratify=df.iloc[:,-1], random_state=34)

trdf = xtr.join(ytr)
tedf = xte.join(yte)

trdf.to_csv('../data/spi10_nrs_train.csv',index=False)
tedf.to_csv('../data/spi10_nrs_test.csv',index=False)

#### Model shooting 

In [1]:
import torch
from dataset import Spi10_2nrs

trdt  = Spi10_2nrs('../data/spi10_nrs_train.csv')
tedt  = Spi10_2nrs('../data/spi10_nrs_test.csv')

trdl  = torch.utils.data.DataLoader(trdt, batch_size=8)
tedl  = torch.utils.data.DataLoader(tedt, batch_size=8)

In [4]:
from models import  SimpleSPI10
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
md = SimpleSPI10().to(device)
loss = nn.CrossEntropyLoss()
opt  = torch.optim.Adam(md.parameters() )

In [6]:
def train(dl,model,lossf,opt):
    for x,y in dl:
        x,y = x.to(device),y.to(device)
        pre = model(x)
        loss = lossf(pre,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

def test(dl,model,lossf):
    model.eval()
    size, acc , losses = len(dl.dataset) ,0,0
    with torch.no_grad():
        for x,y in dl:
            x,y = x.to(device),y.to(device)
            pre = model(x)
            loss = lossf(pre,y)

            acc += (pre.argmax(1)==y.argmax(1)).type(torch.float).sum().item()
            losses += loss.item()
    print(f'{acc/size} : {losses/size}')

from tqdm import tqdm
for _ in range(120):
    train(trdl,md,loss,opt)
test(tedl,md,loss)

0.782608695652174 : 0.22905760161254718


In [23]:
size, acc , losses = len(tedl.dataset) ,0,0
for x,y in tedl:
    x,y = x.to(device),y.to(device)
    pre = md(x)
    lossv = loss(pre,y)

    acc += (pre.argmax(1)==y.argmax(1)).type(torch.float).sum().item()
    print('pre:',pre.argmax(1))
    print('prec:',y.argmax(1))
    losses += lossv.item()

pre: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
pre: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
pre: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
pre: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
pre: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
pre: tensor([0, 0, 0, 0, 0, 0], device='cuda:0')
acc: tensor([0, 0, 0, 0, 0, 0], device='cuda:0')


### PPG (wav2vec) 

#### data split 

In [2]:
import pandas as pd

path='../data/vital2/dd_all_simple.csv'
df = pd.read_csv(path)

In [6]:
df

,key,pacu_nrs
0,JTOR3_210329_13,3
1,JTOR2_211018_13,3
2,JTOR3_201209_11,0
3,JTOR4_210927_14,3
4,JTOR3_210726_08,3
...,...,...
109,JTOR2_210419_13,3
110,JTOR2_211110_09,1
111,JTOR3_211222_11,3
112,JTOR5_201211_08,1


In [ ]:
train

In [7]:
from dataset import nrs2class 

df.pacu_nrs = df.pacu_nrs.map(lambda x: nrs2class(x,pain_type=2))

from sklearn.model_selection import train_test_split

In [25]:
xtr,xte,ytr,yte = train_test_split(df[['key']],df[['pacu_nrs']],test_size=0.4,stratify=df['pacu_nrs'])

In [20]:
# len(xtr),len(xte)
# (68, 46)

# yte.value_counts()
# 0    38
# 1     8
# Name: pacu_nrs

In [27]:
trdf = xtr.join(ytr)
tedf = xte.join(yte)

trdf.to_csv('../data/vital2/dd_all_simple_train.csv',index=False)
tedf.to_csv('../data/vital2/dd_all_simple_test.csv',index=False)

In [29]:
import torch
from dataset import PPGDT

trdt  = PPGDT(op_path= '../data/vital2/dd_all_simple_train.csv')
tedt  = PPGDT(op_path= '../data/vital2/dd_all_simple_test.csv')

trdl  = torch.utils.data.DataLoader(trdt, batch_size=4)
tedl  = torch.utils.data.DataLoader(tedt, batch_size=4)

In [30]:
trit = iter(trdl)

In [31]:
x,y = next (trit)